<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 


## 02 - การจัดการข้อมูล (Data Manipulation) ##

**สารบัญ**

โน้ตบุ๊กนี้จะสำรวจพื้นฐานของการได้มาซึ่งข้อมูลและการจัดการข้อมูลโดยใช้ DataFrame API ซึ่งครอบคลุมเทคนิคที่จำเป็นสำหรับการจัดการและประมวลผลชุดข้อมูล โน้ตบุ๊กนี้ครอบคลุมส่วนต่าง ๆ ดังนี้:

1.  [ที่มาของข้อมูล (Data Background)](#Data-Background)
2.  [cuDF และ pandas](#cuDF-and-pandas)
    * [pandas](#pandas)
    * [cuDF](#cuDF)
3.  [การได้มาซึ่งข้อมูล (Data Acquisition)](#Data-Acquisition)
4.  [การสำรวจข้อมูลเบื้องต้น (Initial Data Exploration)](#Initial-Data-Exploration)
5.  [การจัดทำดัชนีและการเลือกข้อมูลด้วย `.loc` Accessor](#Indexing-and-Data-Selection-with-.loc-Accessor)
6.  [การดำเนินการพื้นฐาน (Basic Operations)](#Basic-Operations)
    * [แบบฝึกหัดที่ 1 - แปลงคอลัมน์ `county` เป็น Title Case](#Exercise-#1---Convert-county-Column-to-Title-Case)
7.  [การรวมข้อมูล (Aggregation)](#Aggregation)
8.  [การใช้ฟังก์ชันที่ผู้ใช้กำหนด (UDFs) ด้วย `.map()` และ `.apply()`](#Applying-User-Defined-Functions-(UDFs)-with-.map()-and-.apply())
9.  [การกรองข้อมูลด้วย `.loc` และ Boolean Mask](#Filtering-with-.loc-and-Boolean-Mask)
    * [แบบฝึกหัดที่ 2 - มณฑลทางเหนือของ Sunderland](#Exercise-#2---Counties-North-of-Sunderland)
10. [การสร้างคอลัมน์ใหม่ (Creating New Columns)](#Creating-New-Columns)
11. [pandas vs. cuDF](#pandas-vs.-cuDF)
12. [cuDF pandas](#cuDF-pandas)
    * [แบบฝึกหัดที่ 3 - การเร่งความเร็วอัตโนมัติ (Automatic Acceleration)](#Exercise-#3---Automatic-Acceleration)


## ข้อมูลเบื้องหลัง ##

สำหรับเวิร์กช็อปนี้ เราจะอ่านข้อมูลเกือบ **60 ล้านรายการ** (ซึ่งเทียบเท่ากับประชากรทั้งหมดของอังกฤษและเวลส์) โดยข้อมูลเหล่านี้ถูกสังเคราะห์ขึ้นจากข้อมูลสำมะโนประชากรอย่างเป็นทางการของสหราชอาณาจักร


## cuDF และ pandas ##


### pandas ###

[pandas](https://pandas.pydata.org/) เป็นไลบรารีโอเพนซอร์สที่ใช้กันอย่างแพร่หลายสำหรับการจัดการและวิเคราะห์ข้อมูลใน Python มีโครงสร้างข้อมูลและเครื่องมือที่มีประสิทธิภาพสูง ใช้งานง่าย สำหรับการทำงานกับข้อมูลที่มีโครงสร้าง เป็นที่นิยมในคำว่า DataFrame ซึ่งเป็นโครงสร้างข้อมูลสำหรับการคำนวณทางสถิติ ในวิทยาการข้อมูล pandas ถูกใช้สำหรับ:

* **การโหลดและเขียนข้อมูล**: อ่านและเขียนจากรูปแบบไฟล์ต่างๆ เช่น CSV, Excel, JSON และฐานข้อมูล SQL
* **การทำความสะอาดและประมวลผล/เตรียมข้อมูล**: ช่วยผู้ใช้ในการจัดการข้อมูลที่ขาดหายไป การรวมชุดข้อมูล และการปรับรูปร่างข้อมูล
* **การวิเคราะห์ข้อมูล**: ดำเนินการจัดกลุ่ม, การรวมข้อมูล และการดำเนินการทางสถิติ

**หมายเหตุ**: การเตรียมข้อมูล (Data preprocessing) หมายถึงกระบวนการแปลงข้อมูลดิบให้อยู่ในรูปแบบที่เหมาะสมกับการวิเคราะห์และงานอื่น ๆ ที่จะตามมา


### cuDF ###

ในทำนองเดียวกัน [cuDF](https://docs.rapids.ai/api/cudf/stable/) เป็นไลบรารี Python GPU DataFrame สำหรับการโหลด, การรวม, การรวมกลุ่ม, การกรอง และการจัดการข้อมูลอื่น ๆ cuDF ได้รับการออกแบบมาเพื่อเร่งเวิร์กโฟลว์วิทยาการข้อมูลโดยใช้ประโยชน์จากพลังการประมวลผลแบบขนานของ GPU ซึ่งอาจให้ความเร็วที่เพิ่มขึ้นอย่างมีนัยสำคัญเมื่อเทียบกับทางเลือกที่ใช้ CPU สำหรับชุดข้อมูลขนาดใหญ่ คุณสมบัติหลักของ cuDF ได้แก่:

* **การเร่งความเร็วด้วย GPU**: ใช้ประโยชน์จาก NVIDIA GPU เพื่อการประมวลผลและวิเคราะห์ข้อมูลที่รวดเร็ว
* **API ที่คล้าย pandas**: มอบอินเทอร์เฟซที่คุ้นเคยให้กับผู้ใช้และการเปลี่ยนไปใช้การประมวลผลบน GPU
* **การรวมเข้ากับไลบรารี RAPIDS อื่น ๆ**: ทำงานร่วมกันได้อย่างราบรื่นกับเครื่องมือเร่งความเร็วด้วย GPU อื่น ๆ ในระบบนิเวศของ RAPIDS


**หมายเหตุ**: ทั้ง Pandas และ cuDF มีวัตถุประสงค์คล้ายกันในการจัดการและวิเคราะห์ข้อมูล แต่ cuDF ได้รับการปรับแต่งเป็นพิเศษสำหรับการเร่งความเร็วด้วย GPU ทำให้มีประโยชน์อย่างยิ่งสำหรับการทำงานกับชุดข้อมูลขนาดใหญ่ที่ประสิทธิภาพเป็นสิ่งสำคัญ

## การได้มาซึ่งข้อมูล (Data Acquisition) ##

ในบริบทของเรา การได้มาซึ่งข้อมูลหมายถึงกระบวนการรวบรวมและนำเข้าข้อมูลจากแหล่งต่าง ๆ เข้าสู่สภาพแวดล้อม Python เพื่อการวิเคราะห์ ประมวลผล และจัดการข้อมูล ข้อมูลสามารถมาจากแหล่งที่หลากหลาย:

* ไฟล์ภายในเครื่องในรูปแบบต่าง ๆ
* ฐานข้อมูล
* API
* การดึงข้อมูลจากเว็บ (Web scraping)

เป็นที่น่าสังเกตว่า ระบบนิเวศไลบรารีที่หลากหลายของ Python ทำให้มีความสามารถรอบด้านในการดึงข้อมูลจากแหล่งต่าง ๆ ทำให้นักวิทยาศาสตร์ข้อมูลสามารถทำงานกับชุดข้อมูลที่หลากหลายได้อย่างมีประสิทธิภาพ การประมวลผลด้วย CPU จะรับผิดชอบในการดึงข้อมูลจาก API หรือการดึงข้อมูลจากเว็บ ในกรณีส่วนใหญ่ แบนด์วิดท์เครือข่ายน่าจะเป็นคอขวด ยิ่งไปกว่านั้น cuDF ไม่มีวิธีในการดึงข้อมูลธุรกรรมจากฐานข้อมูล SQL เข้าสู่หน่วยความจำ GPU ได้โดยตรง แนวทางที่แนะนำสำหรับการอ่านข้อมูลจากฐานข้อมูลคือการใช้วิธีการที่ใช้ CPU ก่อน (เช่น pandas) จากนั้นแปลงเป็น cuDF สำหรับการประมวลผลที่เร่งความเร็วด้วย GPU

ด้านล่างนี้ เราใช้คำสั่ง `head` ของ Linux เพื่อแสดงส่วนเริ่มต้นของไฟล์ข้อมูล ซึ่งช่วยให้เราเข้าใจวิธีการอ่านข้อมูลได้อย่างถูกต้อง

In [ ]:
# DO NOT CHANGE THIS CELL
!head -n 5 data/uk_pop.csv

หนึ่งแถวจะแสดงถึงหนึ่งคน เรามีข้อมูลเกี่ยวกับ `age` (อายุ), `sex` (เพศ), `county` (มณฑล), ตำแหน่งที่ตั้ง (location) และ `name` (ชื่อ) ของพวกเขา ด้วย cuDF ซึ่งเป็น RAPIDS API ที่ให้ GPU-accelerated DataFrame เราสามารถอ่านข้อมูลจาก [รูปแบบที่หลากหลาย](https://rapidsai.github.io/projects/cudf/en/0.10.0/api.html#module-cudf.io.csv) รวมถึง csv, json, parquet, feather, orc และ pandas DataFrames เป็นต้น

In [ ]:
# DO NOT CHANGE THIS CELL
import cudf
import cupy as cp
import numpy as np

from datetime import datetime
import random
import time

ด้านล่างนี้ เราจะอ่านข้อมูลจากไฟล์ CSV ในเครื่องโดยตรงไปยังหน่วยความจำ GPU ด้วยฟังก์ชัน `read_csv()`

In [ ]:
# DO NOT CHANGE THIS CELL
start=time.time()
df=cudf.read_csv('./data/uk_pop.csv')
print(f'Duration: {round(time.time()-start, 2)} seconds')

**หมายเหตุ**: เนื่องจากการจัดการหน่วยความจำ GPU ที่ซับซ้อนเบื้องหลังใน cuDF การโหลดข้อมูลครั้งแรกเข้าสู่สภาพแวดล้อมหน่วยความจำ RAPIDS ที่เพิ่งเริ่มต้นใหม่บางครั้งจะช้ากว่าการโหลดครั้งถัดไปอย่างมาก [RAPIDS Memory Manager](https://github.com/rapidsai/rmm) กำลังเตรียมหน่วยความจำเพิ่มเติมเพื่อรองรับการดำเนินการวิทยาการข้อมูลที่หลากหลายที่เราอาจสนใจใช้กับข้อมูล แทนที่จะจัดสรรและยกเลิกการจัดสรรหน่วยความจำซ้ำๆ ตลอดเวิร์กโฟลว์

ด้านล่างนี้ เราจะได้รับข้อมูลทั่วไปเกี่ยวกับ DataFrame ด้วยเมธอด `DataFrame.info()`

In [ ]:
# DO NOT CHANGE THIS CELL
df.info(memory_usage='deep')

**DataFrame** คือโครงสร้างข้อมูลสองมิติที่มีป้ายกำกับ จัดเรียงเป็นแถวและคอลัมน์ คล้ายกับสเปรดชีตหรือตาราง SQL ทั้งแถวและคอลัมน์มีป้ายกำกับ โดยทั่วไปแถวจะถูกกำกับด้วยดัชนี ในขณะที่คอลัมน์มีชื่อคอลัมน์ ข้อมูลจะถูกจัดเรียงตามป้ายกำกับแถวและคอลัมน์เมื่อทำการดำเนินการ ซึ่งมีประโยชน์สำหรับการเปิดใช้งานการดำเนินการแบบเวกเตอร์ที่มีประสิทธิภาพสูงทั่วทั้งคอลัมน์หรือแถว **Series** หมายถึงอาร์เรย์หนึ่งมิติและโดยทั่วไปจะเกี่ยวข้องกับคอลัมน์เดียวของข้อมูลที่มีดัชนี

มีข้อมูลประมาณ 60 ล้านรายการใน 6 คอลัมน์ cuDF สามารถอ่านข้อมูลจากไฟล์ในเครื่องโดยตรงไปยัง GPU ได้อย่างมีประสิทธิภาพมาก โดยค่าเริ่มต้น cuDF จะสุ่มตัวอย่างชุดข้อมูลเพื่ออนุมานประเภทข้อมูลที่เหมาะสมที่สุดสำหรับแต่ละคอลัมน์

**หมายเหตุ**: DataFrame มีคุณลักษณะ `.dtypes` และ `.columns` ที่สามารถใช้เพื่อรับข้อมูลที่คล้ายกันได้


## การสำรวจข้อมูลเบื้องต้น ##

เมื่อเราโหลดข้อมูลเข้ามาแล้ว มาลองสำรวจข้อมูลเบื้องต้นกัน

ด้านล่างนี้เราจะดูตัวอย่าง DataFrame ด้วยเมธอด `DataFrame.head()`

In [ ]:
# DO NOT CHANGE THIS CELL
df.head()


## การจัดทำดัชนีและการเลือกข้อมูลด้วย `.loc` Accessor ##

`.loc` accessor ใน cuDF DataFrames ใช้สำหรับการทำดัชนีและการเลือกข้อมูลโดยใช้ชื่อกำกับ (label-based) ซึ่งช่วยให้เราสามารถเข้าถึงและจัดการข้อมูลใน DataFrame โดยอิงตามชื่อกำกับของแถวและคอลัมน์ เราสามารถใช้ `DataFrame.loc[row_label(s), column_label(s)]` เพื่อเข้าถึงกลุ่มของแถวและคอลัมน์ เมื่อเลือกชื่อกำกับหลายชื่อ จะใช้ลิสต์ (`[]`) นอกจากนี้ เรายังสามารถใช้ตัวดำเนินการ slice (`:`, เช่น `start:end`) เพื่อระบุช่วงขององค์ประกอบได้

In [ ]:
# DO NOT CHANGE THIS CELL
# get first cell
display(df.loc[0, 'age'])
print('-'*40)

# get multiple rows and columns
display(df.loc[[0, 1, 2], ['age', 'sex', 'county']])
print('-'*40)

# slice a range of rows and columns
display(df.loc[0:5, 'age':'county'])
print('-'*40)

# slice a range of rows and columns
display(df.loc[:10, :'name'])


**หมายเหตุ**: `df[column_label(s)]` เป็นอีกวิธีหนึ่งในการเข้าถึงคอลัมน์ที่ต้องการ คล้ายกับ `df.loc[:, column_labels]`

## การดำเนินการพื้นฐาน (Basic Operations) ##

cuDF รองรับการดำเนินการที่หลากหลายสำหรับข้อมูลตัวเลข แม้ว่าสตริงจะไม่ใช่ชนิดข้อมูลที่เกี่ยวข้องกับ GPU ตามปกติ แต่ cuDF ก็รองรับการดำเนินการสตริงที่เร่งความเร็วอย่างมีประสิทธิภาพ
* การดำเนินการตัวเลข:
    * การดำเนินการทางคณิตศาสตร์: การบวก, การลบ, การคูณ, การหาร
* การดำเนินการสตริง:
    * การแปลงตัวพิมพ์: `.upper()`, `.lower()`, `.title()`
    * การจัดการสตริง: การเชื่อมต่อ, สตริงย่อย, การแยก, การเติม
    * การจับคู่รูปแบบ: `contains()`
    * การแยก: `.split()`
* การดำเนินการเปรียบเทียบ: มากกว่า, น้อยกว่า, เท่ากับ, เป็นต้น

การดำเนินการเหล่านี้จะถูกดำเนินการทีละองค์ประกอบสำหรับแต่ละแถว สิ่งนี้ช่วยให้สามารถดำเนินการ **vectorized operations** ได้อย่างมีประสิทธิภาพทั่วทั้งคอลัมน์ การดำเนินการเหล่านี้ถูกนำไปใช้เป็นการดำเนินการแบบเวกเตอร์แทนการวนซ้ำ เนื่องจากการดำเนินการแบบเวกเตอร์สามารถนำไปใช้กับอาร์เรย์ข้อมูลทั้งหมดได้ แทนที่จะวนซ้ำแต่ละองค์ประกอบทีละรายการ การสร้างเวกเตอร์เร็วกว่าการวนซ้ำแต่ละองค์ประกอบอย่างมาก โดยเฉพาะอย่างยิ่งสำหรับชุดข้อมูลขนาดใหญ่ เมื่อดำเนินการกับหลายคอลัมน์ การดำเนินการจะถูกจัดเรียงตามดัชนี ทำให้มั่นใจได้ว่าการคำนวณจะดำเนินการกับองค์ประกอบที่ถูกต้องที่สอดคล้องกันทั่วทั้งคอลัมน์ การดำเนินการทีละองค์ประกอบเหล่านี้มักจะได้รับการปรับให้เหมาะสมอย่างสูงและสามารถเร็วกว่าการวนซ้ำแบบชัดเจนมาก โดยเฉพาะอย่างยิ่งสำหรับชุดข้อมูลขนาดใหญ่ เราสามารถรับอาร์เรย์ข้อมูลพื้นฐานได้ด้วยคุณสมบัติ `.values` ซึ่งมีประโยชน์เมื่อเราต้องการดำเนินการกับข้อมูลพื้นฐาน

**หมายเหตุ**: ไม่รองรับการวนซ้ำบน cuDF Series, DataFrame หรือ Index เนื่องจาก การวนซ้ำข้อมูลที่อยู่บน GPU จะให้ประสิทธิภาพที่แย่มาก เนื่องจาก GPU ได้รับการปรับให้เหมาะสมสำหรับการดำเนินการแบบขนานสูงมากกว่าการดำเนินการแบบลำดับ

ด้านล่างนี้เราจะคำนวณปีเกิดของแต่ละบุคคล

In [ ]:
# DO NOT CHANGE THIS CELL
# get current year
current_year=datetime.now().year

# derive the birth year
display(current_year-df.loc[:, 'age'])

# get the age array (CuPy for cuDF)
age_ary=df.loc[:, 'age'].values

# derive the birth year
current_year-age_ary

เมื่อดำเนินการระหว่าง DataFrame กับค่าสเกลาร์ ค่าสเกลาร์จะถูก "broadcast" เพื่อให้ตรงกับรูปร่างของ DataFrame ซึ่งจะนำไปใช้กับแต่ละองค์ประกอบโดยอัตโนมัติ



```
current_year - df.loc[:, 'age']
-------------------------------
  (scalar)          (array)    
    2024,    -         0
    2024,    -         0
    2024,    -         0
    2024,    -         0
    2024,    -         0
    ...      -         ...
```

นี่เป็นส่วนหนึ่งที่อธิบายว่าทำไม cuDF จึงให้ประสิทธิภาพที่ดีขึ้นอย่างมากเมื่อเทียบกับ pandas โดยเฉพาะอย่างยิ่งสำหรับชุดข้อมูลขนาดใหญ่ สถาปัตยกรรมการประมวลผลแบบขนานของ GPU ถูกออกแบบมาพร้อมกับคอร์ขนาดเล็กและเฉพาะทางหลายพันคอร์ที่สามารถดำเนินการหลายคำสั่งพร้อมกันได้ สถาปัตยกรรมนี้เหมาะอย่างยิ่งสำหรับการดำเนินการแบบเวกเตอร์ ซึ่งจะดำเนินการคำสั่งเดียวกันกับองค์ประกอบข้อมูลหลายรายการพร้อมกัน


<a name='s4.1'></a>
### แบบฝึกหัดที่ 1 - แปลงคอลัมน์ `county` เป็น Title Case ###

จากข้อมูลที่มีอยู่ ทุกมณฑลเป็นตัวพิมพ์ใหญ่ทั้งหมด เราต้องการแปลงคอลัมน์ `county` ให้เป็น Title Case (ตัวอักษรแรกของแต่ละคำเป็นตัวพิมพ์ใหญ่)

**คำแนะนำ**:

* แก้ไขเฉพาะส่วน `<FIXME>` และรันเซลล์ด้านล่างเพื่อแปลงคอลัมน์ `county` เป็น Title Case

In [ ]:
df['county'].str.<<<<FIXME>>>>

คลิก ... เพื่อดูเฉลย

การดำเนินการเปรียบเทียบหรือการใช้เงื่อนไขจะสร้างค่าบูลีน (True/False) ที่สัมพันธ์กันแบบทีละองค์ประกอบ

ด้านล่างนี้ เราจะตรวจสอบว่าแต่ละคนเป็นผู้ใหญ่หรือไม่


In [ ]:
# DO NOT CHANGE THIS CELL
df['age']>=18


## การรวมข้อมูล (Aggregation) ##

การรวมข้อมูลเป็นกระบวนการที่สำคัญสำหรับงานด้านวิทยาศาสตร์ข้อมูล ซึ่งช่วยให้เราสามารถสรุปและวิเคราะห์ข้อมูลที่จัดกลุ่มได้ มักใช้สำหรับงานต่างๆ เช่น การคำนวณยอดรวม, ค่าเฉลี่ย, จำนวนนับ เป็นต้น cuDF รองรับการรวมข้อมูลทั่วไป เช่น `.sum()`, `.mean()`, `.min()`, `.max()`, `.count()`, `.std()` (ค่าเบี่ยงเบนมาตรฐาน) เป็นต้น นอกจากนี้ยังรองรับการรวมข้อมูลขั้นสูงเพิ่มเติม เช่น `.quantile()` และ `.corr()` (ความสัมพันธ์) ด้วยพารามิเตอร์ `axis` การดำเนินการรวมข้อมูลสามารถนำไปใช้กับคอลัมน์ (`0`) หรือแถว (`1`) ได้

เมื่อการรวมข้อมูลถูกนำมาใช้เป็นการดำเนินการแบบเวกเตอร์ โดยเฉพาะการดำเนินการแบบลดขนาด (reduction operation) จะมีประสิทธิภาพสูงบน GPU เนื่องจากสามารถประมวลผลองค์ประกอบข้อมูลจำนวนมากพร้อมกันและแบบขนานได้ การดำเนินการตามคอลัมน์ยังได้รับประโยชน์จาก [Apache Arrow columnar memory format](https://arrow.apache.org/docs/format/Columnar.html)

<p><img src='images/sum_reduction.png' width=720></p>

ด้านล่างนี้ เราจะคำนวณค่าเฉลี่ยเลขคณิตของ `lat` และ `long` เพื่อหาจุดศูนย์กลางโดยประมาณ

In [ ]:
# DO NOT CHANGE THIS CELL
df[['lat', 'long']].mean()


## การใช้ฟังก์ชันที่ผู้ใช้กำหนด (UDFs) ด้วย `.map()` และ `.apply()` ##

เมธอด `.map()` และ `.apply()` เป็นวิธีหลักในการใช้ฟังก์ชันที่ผู้ใช้กำหนด (user-defined functions) แบบทีละองค์ประกอบ และแบบตามแถวหรือคอลัมน์ ตามลำดับ เราสามารถส่งฟังก์ชันที่เรียกใช้งานได้ (ทั้งแบบ built-in หรือที่ผู้ใช้กำหนด) เป็นอาร์กิวเมนต์ ซึ่งจะถูกนำไปใช้กับโครงสร้างข้อมูลทั้งหมด ไม่ใช่ทุกการดำเนินการจะสามารถแปลงเป็นเวกเตอร์ได้ โดยเฉพาะตรรกะที่กำหนดเองที่ซับซ้อน ในกรณีเช่นนี้ อาจจำเป็นต้องใช้เมธอดเช่น `.apply()` หรือ UDFs ที่กำหนดเอง

ด้านล่างนี้ เราจะใช้ `.apply()` เพื่อตรวจสอบว่าแต่ละคนเป็นผู้ใหญ่หรือไม่

In [ ]:
# DO NOT CHNAGE THIS CELL
# define a function to check if age is greater than or equal to 18
start=time.time()
def is_adult(row): 
    if row['age']>=18: 
        return 1
    else: 
        return 0

# derive the birth year
display(df.apply(is_adult, axis=1))
print(f'Duration: {round(time.time()-start, 2)} seconds')

เรายังสามารถใช้ [**lambda function**](https://docs.python.org/3/glossary.html#term-lambda) ได้เมื่อฟังก์ชันนั้นเรียบง่าย Lambda function ถูกจำกัดให้มีเพียงนิพจน์เดียว แต่สามารถรวมคำสั่งเงื่อนไขและอาร์กิวเมนต์หลายตัวได้

In [ ]:
# DO NOT CHANGE THIS CELL
# derive the birth year
start=time.time()
display(df.apply(lambda x: 1 if x['age']>=18 else 0, axis=1))
print(f'Duration: {round(time.time()-start, 2)} seconds')


**หมายเหตุ**: ฟังก์ชัน `.apply()` ใน pandas รองรับฟังก์ชันที่ผู้ใช้กำหนดเองได้ ซึ่งสามารถรวมการดำเนินการใดๆ ที่นำไปใช้กับแต่ละค่าของ Series และ DataFrame ได้ cuDF ก็รองรับ `.apply()` เช่นกัน แต่จะอาศัย Numba ในการคอมไพล์ JIT UDF (ไม่ได้อยู่ในขอบเขต) และรันบน GPU ซึ่งสามารถทำได้อย่างรวดเร็วมาก แต่ก็มีข้อจำกัดบางประการเกี่ยวกับการดำเนินการที่อนุญาตใน UDF ดูรายละเอียดได้จากเอกสารเกี่ยวกับ [UDFs](https://docs.rapids.ai/api/cudf/stable/user_guide/guide-to-udfs/)

In [ ]:
# DO NOT CHANGE THIS CELL
# derive the birth year
start=time.time()
display((df['age']>=18).astype('int'))
print(f'Duration: {round(time.time()-start, 2)} seconds')

ด้านล่างนี้ เราใช้ `Series.map()` เพื่อหาจำนวนตัวอักษรในชื่อของแต่ละคน

In [ ]:
# DO NOT CHANGE THIS CELL
df['name'].map(lambda x: len(x))


## การกรองข้อมูลด้วย `.loc` และ Boolean Mask ##

Boolean mask คืออาร์เรย์ของค่า `True`/`False` ที่สอดคล้องกับอาร์เรย์หรือโครงสร้างข้อมูลอื่น ๆ แบบทีละองค์ประกอบ ใช้สำหรับการกรองและเลือกข้อมูลตามเงื่อนไขบางอย่าง ในบริบทนี้ mask สามารถใช้เพื่อทำดัชนีหรือกรอง DataFrame ด้วย `.loc` โดยเลือกเฉพาะองค์ประกอบที่ mask เป็น `True`

**หมายเหตุ**: การใช้ Boolean masking มักจะมีประสิทธิภาพมากกว่าวิธีการแบบวนซ้ำ โดยเฉพาะอย่างยิ่งสำหรับชุดข้อมูลขนาดใหญ่ เนื่องจากใช้ประโยชน์จากการดำเนินการแบบเวกเตอร์

ด้านล่างนี้ เราจะใช้ `.loc` accessor และ Boolean mask เพื่อกรองผู้ที่มีชื่อขึ้นต้นด้วยตัวอักษร `E`

In [ ]:
# DO NOT CHANGE THIS CELL
boolean_mask=df['name'].str.startswith('E')
df.loc[boolean_mask]

สามารถรวมเงื่อนไขหลายข้อได้โดยใช้ตัวดำเนินการตรรกะ (`&` และ `|`)

**หมายเหตุ**: เมื่อใช้หลายเงื่อนไข สิ่งสำคัญคือต้องครอบแต่ละเงื่อนไขด้วยวงเล็บ (`(` และ `)`) เพื่อให้แน่ใจว่าลำดับการดำเนินการถูกต้อง

ด้านล่างนี้ เราจะใช้ `.loc` accessor และเงื่อนไขหลายข้อเพื่อกรองผู้ใหญ่ที่มีชื่อขึ้นต้นด้วยตัวอักษร `E`

In [ ]:
# DO NOT CHANGE THIS CELL
df[(df['age']>=18) | (df['name'].str.startswith('E'))]

<a name='s4.1'></a>
### แบบฝึกหัดที่ 2 - มณฑลทางเหนือของ Sunderland ###

แบบฝึกหัดนี้จะต้องใช้ `.loc` accessor และเทคนิคหลายอย่างที่อธิบายไว้ข้างต้น เราต้องการระบุละติจูดของผู้อยู่อาศัยที่อยู่เหนือสุดของมณฑล Sunderland (บุคคลที่มีค่า `lat` สูงสุด) จากนั้นพิจารณาว่ามณฑลใดบ้างที่มีผู้อยู่อาศัยอยู่ทางเหนือของผู้อยู่อาศัยคนนี้ ใช้เมธอด `Series.unique()` เพื่อลบค่าที่ซ้ำกันออกจากผลลัพธ์

**คำแนะนำ**:
* แก้ไขเฉพาะส่วน `<FIXME>` และรันเซลล์ด้านล่างเพื่อระบุมณฑลที่อยู่ทางเหนือของ Sunderland

In [ ]:
sunderland_residents=df.loc[<<<<FIXME>>>>]
northmost_sunderland_lat=sunderland_residents['lat'].max()
df.loc[df['lat'] > northmost_sunderland_lat]['county'].unique()

คลิก ... เพื่อดูเฉลย

## การสร้างคอลัมน์ใหม่ ##

เราสามารถสร้างคอลัมน์ใหม่ได้โดยการกำหนดค่าให้กับชื่อคอลัมน์ คอลัมน์ใหม่ควรมีจำนวนแถวเท่ากับ DataFrame ที่มีอยู่ โดยทั่วไป เราจะสร้างคอลัมน์ใหม่โดยการดำเนินการกับคอลัมน์ที่มีอยู่แล้ว

ด้านล่างนี้ เราจะสร้างคอลัมน์เพิ่มเติมอีกสองสามคอลัมน์

In [ ]:
# DO NOT CHANGE THIS CELL
# get current year
current_year=datetime.now().year

# numerical operations
df['birth_year']=current_year-df['age']

# string operations
df['sex_normalize']=df['sex'].str.upper()
df['county_normalize']=df['county'].str.title().str.replace(' ', '_')
df['name']=df['name'].str.title()

# preview
df.head()

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## pandas vs. cuDF ##

นอกเหนือจากประสิทธิภาพที่ดีกว่ามากกับชุดข้อมูลขนาดใหญ่แล้ว cuDF มีหน้าตาและความรู้สึกคล้ายกับ Pandas มาก เพื่อทบทวน cuDF และ pandas มีความคล้ายคลึงกันดังนี้:

* **ความคล้ายคลึงของ API**: cuDF มี API ที่คล้ายกับ pandas ซึ่งคุ้นเคยกับวิศวกรข้อมูลและนักวิทยาศาสตร์ข้อมูล มีเป้าหมายที่จะใช้ฟังก์ชันและการดำเนินการหลายอย่างเช่นเดียวกับ pandas ทำให้ผู้ใช้สามารถเร่งเวิร์กโฟลว์ pandas ที่มีอยู่ได้อย่างง่ายดาย
* **การดำเนินการที่คล้ายกัน**: cuDF ใช้การดำเนินการ pandas ทั่วไปหลายอย่าง เช่น การกรอง, การเชื่อม, การรวม และ groupby

<p><img src='images/pandas_vs_cudf.png' width=1080></p>

เมื่อเปรียบเทียบกับ pandas cuDF มักจะทำงานได้ดีกว่าสำหรับชุดข้อมูลขนาดใหญ่เนื่องจากคุณสมบัติดังต่อไปนี้:

* GPU มีความโดดเด่นในการคำนวณแบบขนาน ซึ่งเป็นข้อได้เปรียบสำหรับงานวิทยาศาสตร์ข้อมูลและ Machine Learning หลายอย่าง
* โดยทั่วไป GPU มีแบนด์วิดท์หน่วยความจำสูงกว่า CPU มาก ทำให้เข้าถึงข้อมูลได้เร็วขึ้นในการดำเนินการที่จำกัดด้วยหน่วยความจำ
* cuDF ใช้ประโยชน์จากความสามารถของ GPU ในการดำเนินการแบบเวกเตอร์ได้อย่างมีประสิทธิภาพ ซึ่งเป็นประโยชน์อย่างยิ่งสำหรับชุดข้อมูลขนาดใหญ่
* cuDF ใช้รูปแบบข้อมูลแบบคอลัมน์ ซึ่งสามารถนำไปสู่รูปแบบการเข้าถึงหน่วยความจำที่มีประสิทธิภาพมากขึ้นบน GPU เมื่อดำเนินการกับ cuDF Dataframes การดำเนินการกับคอลัมน์มักจะมีประสิทธิภาพดีกว่าการดำเนินการตามแถวมาก

**หมายเหตุ**: สิ่งสำคัญคือต้องทราบว่าข้อได้เปรียบด้านประสิทธิภาพของ cuDF เหนือ pandas อาจแตกต่างกันไปขึ้นอยู่กับการดำเนินการเฉพาะ ขนาดข้อมูล และการกำหนดค่าฮาร์ดแวร์ สำหรับชุดข้อมูลขนาดเล็กหรือการดำเนินการที่ง่ายกว่า ค่าใช้จ่ายในการเริ่มต้น GPU อาจทำให้ pandas บน CPU เร็วขึ้น

## cuDF pandas ##

เริ่มตั้งแต่เวอร์ชัน `23.10.01` เป็นต้นไป cuDF ได้เปิดตัว **โหมดตัวเร่ง pandas** (`cudf.pandas`) ที่รองรับ 100% ของ pandas API โหมดนี้ช่วยให้ผู้ใช้สามารถเร่งโค้ด pandas บน GPU โดยไม่ต้องเปลี่ยนแปลงโค้ดใดๆ อย่างไรก็ตาม ไม่ใช่ทุกการดำเนินการจะสามารถทำงานบน GPU ได้ เมื่อใช้ `cudf.pandas` การดำเนินการที่สามารถเร่งความเร็วได้จะรันบน GPU ในขณะที่การดำเนินการที่ไม่รองรับจะกลับไปทำงานบน pandas บน CPU โดยอัตโนมัติ ตัวอย่างเช่น `.read_sql()` จะอ่าน SQL ด้วย pandas ก่อนแล้วจึงย้ายข้อมูลไปยัง cuDF

มีสองวิธีในการเปิดใช้งาน cuDF pandas:
- คำสั่ง Jupyter Magic Command
```
%load_ext cudf.pandas
import pandas
...
```
- Python Import
```
import cudf.pandas
cudf.pandas.install()

import pandas as pd
...
```

**หมายเหตุ**: ไม่จำเป็นต้องมีการเปลี่ยนแปลงอื่นใด ซึ่งมีประโยชน์สำหรับการเร่งปริมาณงานที่มีอยู่ได้อย่างรวดเร็วโดยมีการเปลี่ยนแปลงโค้ดน้อยที่สุด ข้อมูลเพิ่มเติมเกี่ยวกับ cuDF pandas สามารถดูได้ [ที่นี่](https://docs.rapids.ai/api/cudf/stable/cudf_pandas/)

cuDF pandas เป็นตัวเร่งความเร็วที่ไม่ต้องเปลี่ยนแปลงโค้ดสำหรับ pandas เพื่อการเร่งความเร็วอัตโนมัติของการเรียกใช้ pandas ที่รองรับ

ด้านล่างนี้ เราจะรันการดำเนินการ DataFrame พื้นฐานบางอย่างด้วย pandas ก่อนที่จะสาธิตวิธีการเปิดใช้งาน cudf pandas

In [ ]:
# DO NOT CHANGE THIS CELL
# %load_ext cudf.pandas

In [ ]:
# DO NOT CHANGE THIS CELL
import pandas as pd
import time
from datetime import datetime

In [ ]:
# %%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
start=time.time()

df=pd.read_csv('./data/uk_pop.csv')
current_year=datetime.now().year

df['birth_year']=current_year-df['age']

df['sex_normalize']=df['sex'].str.upper()
df['county_normalize']=df['county'].str.title().str.replace(' ', '_')
df['name']=df['name'].str.title()

print(f'Duration: {round(time.time()-start, 2)} seconds')

display(df.head())

<a name='s4.1'></a>
### แบบฝึกหัดที่ 3 - การเร่งความเร็วอัตโนมัติ ###

**คำแนะนำ**:

* ย้อนกลับไปที่ส่วนต้นของหัวข้อย่อยนี้ รันเซลล์ใหม่ และยกเลิกการคอมเมนต์คำสั่ง magic command `%load_ext` เพื่อเร่งความเร็วด้วย cuDF pandas
* สังเกตการเร่งความเร็วที่เกิดขึ้น
* ย้อนกลับไปที่ส่วนต้นของหัวข้อย่อยนี้ รันเซลล์ใหม่ และยกเลิกการคอมเมนต์คำสั่ง magic command `%%cudf.pandas.line_profile` เพื่อใช้ line profiler
* สังเกตผลลัพธ์จาก line profiler

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [โน้ตบุ๊กถัดไป](1-03_memory_management.ipynb) กันเลย